# Exciton Group Theory Analysis Example

This notebook demonstrates the use of the `ExcitonGroupTheory` class for analyzing the symmetry properties of exciton states in crystalline materials using group theory.

## Theoretical Background

### Group Theory for Excitons

Exciton states transform according to the irreducible representations of the little group $G_k$ of the exciton momentum $\mathbf{k}$. The symmetry analysis involves:

1. **Little Group Identification**: Finding all symmetry operations that leave the exciton momentum invariant
2. **Representation Matrix Calculation**: Computing how exciton states transform under symmetry operations
3. **Character Analysis**: Determining the trace of representation matrices
4. **Irreducible Representation Decomposition**: Classifying states according to their symmetry properties

### Mathematical Formalism

The representation matrix for symmetry operation $R$ is:

$D^{(n)}_R = \langle\psi_n(R\mathbf{k})| U(R) |\psi_n(\mathbf{k})\rangle$

The character of this representation:

$\chi^{(n)}(R) = \text{Tr}[D^{(n)}_R]$

Irreducible representation decomposition using the reduction formula:
$a_i = \frac{1}{|G|} \sum_{R \in G} \chi^{(R)} \chi_i^{(R)*}$

## What this notebook demonstrates:

1. How to initialize the ExcitonGroupTheory class
2. How to perform group theory analysis for exciton states
3. How to interpret symmetry results and optical selection rules
4. How to save and visualize the analysis results

## Features
- Universal space group support for all 230 space groups
- General symmetry classification using spglib for any crystal system
- Non-symmorphic operations including screw rotations and glide reflections
- Automatic point group identification using spglib
- Irreducible representation decomposition using spgrep
- Optical activity analysis (Raman, IR, electric dipole)
- LaTeX formatting for publication-quality plots

## Requirements
- `spglib` and `spgrep` libraries for symmetry analysis
- Yambo calculation with BSE and electron-phonon data

## Crystal Systems Supported
✅ **Triclinic** (P1, P-1) • ✅ **Monoclinic** (P2, C2/m) • ✅ **Orthorhombic** (Pmmm, Fddd)  
✅ **Tetragonal** (P4, I4/mcm) • ✅ **Trigonal** (P3, R3m) • ✅ **Hexagonal** (P6₃/mmc) • ✅ **Cubic** (Fd3m)

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

from yambopy.optical_properties.exciton_group_theory import ExcitonGroupTheory

# Set up plotting parameters
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Configuration

Set up the paths and parameters for your calculation:

In [ ]:
# Basic input parameters
# Adjust these paths according to your calculation setup
path = './'  # Current directory or path to your calculation
SAVE_dir = './SAVE'
BSE_dir = './GW_BSE/bse'  # or 'GW_BSE' depending on your setup
LELPH_dir = './lelph'  # Directory containing electron-phonon data

# Exciton analysis parameters
iQ = 1  # Exciton Q-point index (1-based, as in Yambo)
nstates = 10  # Number of exciton states to analyze
degen_thres = 0.001  # Degeneracy threshold in eV

# Band range (adjust according to your calculation)
bands_range = [6, 10]  # Example: bands 6 to 10

print("Configuration:")
print(f"  Path: {path}")
print(f"  BSE directory: {BSE_dir}")
print(f"  LELPH directory: {LELPH_dir}")
print(f"  Analyzing Q-point: {iQ}")
print(f"  Number of states: {nstates}")
print(f"  Band range: {bands_range}")
print("-" * 50)

## Initialize ExcitonGroupTheory

The `ExcitonGroupTheory` class reads the necessary database files and sets up the symmetry operations. The initialization process involves:

1. **Database Reading**: Loading lattice, wavefunction, BSE, and electron-phonon databases
2. **Symmetry Setup**: Reading crystallographic symmetry operations
3. **D-matrix Preparation**: Setting up wavefunction rotation matrices
4. **K-point Mapping**: Building k-point search trees for efficient operations

In [ ]:
# Initialize the ExcitonGroupTheory class
print("Initializing ExcitonGroupTheory class...")
print("This will read the database files and set up symmetry operations.")

egt = ExcitonGroupTheory(
    path=path,
    save='SAVE',
    BSE_dir=BSE_dir,
    LELPH_dir=LELPH_dir,
    bands_range=bands_range,
    read_symm_from_ns_db_file=True  # Read symmetries from ns.db1
)

print("\nInitialization completed successfully!")
print(f"  Point group: {egt.point_group_label}")
print(f"  Space group: {egt.spacegroup_label}")
print(f"  Number of symmetry operations: {len(egt.symm_mats)}")
print(f"  Number of IBZ k-points: {egt.nibz}")
print(f"  Number of bands: {egt.nbands}")

## General Symmetry Classification


In [1]:
# Universal symmetry operation classification
print("\n" + "=" * 80)
print("GENERAL SYMMETRY OPERATIONS ANALYSIS")
print("=" * 80)

# Use the general classification method
operations = egt.classify_symmetry_operations()
summary = operations.get('_summary', {})

print(f"\nCRYSTAL STRUCTURE INFORMATION:")
print(f"   Space Group: {summary.get('space_group', 'Unknown')} (#{summary.get('space_group_number', '?')})")
print(f"   Point Group: {summary.get('point_group', 'Unknown')}")
print(f"   Crystal System: {summary.get('crystal_system', 'Unknown').title()}")
print(f"   Total Operations: {summary.get('total_operations', 0)}")

print(f"\n OPERATION BREAKDOWN:")
print("-" * 70)

operation_symbols = {
    'identity': 'E (Identity)',
    'rotation': 'Cₙ (Rotations)',
    'reflection': 'σ (Reflections)',
    'inversion': 'i (Inversion)',
    'rotoinversion': 'Sₙ (Rotoinversions)',
    'screw': 'nₘ (Screw rotations)',
    'glide': 'g (Glide reflections)',
    'unknown': '? (Unclassified)'
}

total_classified = 0
for op_type, op_list in operations.items():
    if op_type == '_summary':
        continue
    if op_list:
        description = operation_symbols.get(op_type, op_type.title())
        count = len(op_list)
        total_classified += count
        print(f"  {description:25s}: {count:2d} operations")

print("-" * 70)
print(f"  Total classified: {total_classified}/{summary.get('total_operations', 0)}")

# Show detailed analysis for key operations
print(f"\n🔬 KEY OPERATIONS DETAILS:")
print("-" * 70)

key_operations = ['identity', 'rotation', 'reflection', 'inversion']
for op_type in key_operations:
    op_list = operations.get(op_type, [])
    if op_list:
        print(f"\n  {operation_symbols.get(op_type, op_type.title())}:")
        for i, op_data in enumerate(op_list[:3]):  # Show first 3 of each type
            if len(op_data) >= 4:
                idx, mat, desc, symbol, spglib_info = op_data
                print(f"    {i+1}. {desc} ({symbol})")
                if spglib_info.get('has_translation', False):
                    trans = spglib_info.get('spg_translation', [0, 0, 0])
                    print(f"       Translation: [{trans[0]:6.3f} {trans[1]:6.3f} {trans[2]:6.3f}]")
        if len(op_list) > 3:
            print(f"    ... and {len(op_list) - 3} more")


GENERAL SYMMETRY OPERATIONS ANALYSIS


NameError: name 'egt' is not defined

## Comprehensive Symmetry Display


In [ ]:
# Run the comprehensive display method
print("\n" + "=" * 80)
print("COMPREHENSIVE SYMMETRY ANALYSIS")
print("=" * 80)

egt.display_symmetry_operations()

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE")
print("=" * 80)
print("\nThis analysis now works for all 230 space groups!")
print("Features:")
print("• General classification using spglib")
print("• Support for non-symmorphic operations (screw, glide)")
print("• Crystallographic standard notation")
print("• Comprehensive crystal system information")

## Exciton Group Theory Analysis

Analyze the exciton states and determine their irreducible representations:

In [ ]:
# Perform the group theory analysis
print(f"Performing group theory analysis for Q-point {iQ}...")
print(f"Analyzing {nstates} exciton states with degeneracy threshold {degen_thres} eV")

results = egt.analyze_exciton_symmetry(
    iQ=iQ, 
    nstates=nstates, 
    degen_thres=degen_thres
)

print("\nAnalysis completed successfully!")
print("\n" + "=" * 60)
print("GROUP THEORY ANALYSIS RESULTS")
print("=" * 60)

# Display basic information
print(f"Q-point coordinates: {results['q_point']}")
print(f"Point group: {results['point_group_label']}")
print(f"Little group size: {len(results['little_group'])}")
print(f"Number of energy levels: {len(results['unique_energies'])}")
print(f"Total exciton states analyzed: {len(results['exciton_energies'])}")

## Detailed Results Analysis

Let's examine the detailed results of the group theory analysis:

In [ ]:
print("\n" + "=" * 60)
print("ENERGY LEVELS AND SYMMETRY CLASSIFICATION")
print("=" * 60)
print(f"{'Level':<6} {'Energy (eV)':<12} {'Degeneracy':<11} {'Irrep':<20} {'Optical Activity'}")
print("-" * 80)

for i, (energy, degen, irrep, activity) in enumerate(zip(
    results['unique_energies'],
    results['degeneracies'],
    results['irrep_decomposition'],
    results['optical_activity']
)):
    # Determine optical activity status
    status = []
    if activity['electric_dipole_allowed']:
        status.append('Optical')
    if activity['raman_active']:
        status.append('Raman')
    if activity['ir_active']:
        status.append('IR')
    
    status_str = ', '.join(status) if status else 'Forbidden'
    
    print(f"{i+1:<6} {energy:<12.4f} {degen:<11} {irrep:<20} {status_str}")

# Display symmetry classes
print("\n" + "=" * 60)
print("SYMMETRY CLASSES")
print("=" * 60)
for i, class_name in enumerate(results['classes']):
    operations = results['class_dict'].get(i, [])
    print(f"{class_name:<10}: {operations}")

## Initialize ExcitonGroupTheory Class

The `ExcitonGroupTheory` class reads the necessary database files and sets up the symmetry operations. The initialization process involves:

1. **Database Reading**: Loading lattice, wavefunction, BSE, and electron-phonon databases
2. **Symmetry Setup**: Reading crystallographic symmetry operations
3. **D-matrix Preparation**: Setting up wavefunction rotation matrices
4. **K-point Mapping**: Building k-point search trees for efficient operations

In [ ]:
# Initialize the ExcitonGroupTheory class
print("Initializing ExcitonGroupTheory class...")
print("This will read the database files and set up symmetry operations.")

egt = ExcitonGroupTheory(
    path=path,
    save='SAVE',
    BSE_dir=BSE_dir,
    LELPH_dir=LELPH_dir,
    bands_range=bands_range,
    read_symm_from_ns_db_file=True  # Read symmetries from ns.db1
)

print("\nInitialization completed successfully!")
print(f"Number of symmetry operations: {len(egt.symm_mats)}")
print(f"Number of IBZ k-points: {egt.nibz}")
print(f"Number of bands: {egt.nbands}")
print(f"Lattice vectors shape: {egt.lat_vecs.shape}")
print(f"Reciprocal lattice vectors shape: {egt.blat_vecs.shape}")

## Perform Group Theory Analysis

The `analyze_exciton_symmetry()` method performs the complete group theory analysis:

### Method Overview

1. **`_determine_little_group()`**: Identifies symmetry operations that leave the Q-point invariant
2. **`_read_exciton_states()`**: Loads BSE eigenvalues and eigenvectors
3. **`_group_degenerate_states()`**: Groups states by energy within the degeneracy threshold
4. **`_compute_representation_matrices()`**: Calculates how states transform under symmetry
5. **`_compute_characters()`**: Computes traces of representation matrices
6. **`_decompose_representations()`**: Decomposes into irreducible representations
7. **`_analyze_optical_activity()`**: Determines optical selection rules

In [ ]:
# Perform the group theory analysis
print(f"Performing group theory analysis for Q-point {iQ}...")
print(f"Analyzing {nstates} exciton states with degeneracy threshold {degen_thres} eV")

results = egt.analyze_exciton_symmetry(
    iQ=iQ,
    nstates=nstates,
    degen_thres=degen_thres
)

print("\nAnalysis completed successfully!")
print("\n" + "=" * 60)
print("GROUP THEORY ANALYSIS RESULTS")
print("=" * 60)

# Display basic information
print(f"Q-point coordinates: {results['q_point']}")
print(f"Point group: {results['point_group_label']}")
print(f"Little group size: {len(results['little_group'])}")
print(f"Number of energy levels: {len(results['unique_energies'])}")
print(f"Total exciton states analyzed: {len(results['exciton_energies'])}")

## Detailed Results Analysis

Let's examine the detailed results of the group theory analysis:

In [ ]:
print("\n" + "=" * 60)
print("ENERGY LEVELS AND SYMMETRY CLASSIFICATION")
print("=" * 60)
print(f"{'Level':<6} {'Energy (eV)':<12} {'Degeneracy':<11} {'Irrep':<20} {'Optical Activity'}")
print("-" * 80)

for i, (energy, degen, irrep, activity) in enumerate(zip(
    results['unique_energies'],
    results['degeneracies'],
    results['irrep_decomposition'],
    results['optical_activity']
)):
    # Determine optical activity status
    status = []
    if activity['electric_dipole_allowed']:
        status.append('Optical')
    if activity['raman_active']:
        status.append('Raman')
    if activity['ir_active']:
        status.append('IR')
    
    status_str = ', '.join(status) if status else 'Forbidden'
    
    print(f"{i+1:<6} {energy:<12.4f} {degen:<11} {irrep:<20} {status_str}")

# Display symmetry classes
print("\n" + "=" * 60)
print("SYMMETRY CLASSES")
print("=" * 60)
for i, class_name in enumerate(results['classes']):
    operations = results['class_dict'].get(i, [])
    print(f"{class_name:<10}: {operations}")

## Optical Selection Rules Analysis

The optical activity analysis determines which transitions are allowed based on symmetry selection rules:

In [ ]:
print("\n" + "=" * 60)
print("OPTICAL SELECTION RULES ANALYSIS")
print("=" * 60)

for i, (energy, irrep, activity) in enumerate(zip(
    results['unique_energies'][:5],  # Show first 5 states
    results['irrep_decomposition'][:5],
    results['optical_activity'][:5]
)):
    print(f"\nExciton State {i+1}: {energy:.4f} eV")
    print(f"Irreducible Representation: {irrep}")
    print("-" * 40)
    
    # Selection rules
    print("Selection Rules:")
    print(f"  Electric Dipole Allowed: {activity['electric_dipole_allowed']}")
    print(f"  Raman Active: {activity['raman_active']}")
    print(f"  IR Active: {activity['ir_active']}")
    
    # Physical interpretation
    print("\nPhysical Interpretation:")
    if activity['electric_dipole_allowed']:
        print("  ✓ Bright exciton - observable in absorption/photoluminescence")
        print("  ✓ Can couple to light via electric dipole transitions")
    else:
        print("  ✗ Dark exciton - forbidden in electric dipole approximation")
        print("  ✗ Not directly observable in linear optical spectroscopy")
    
    if activity['raman_active']:
        print("  ✓ Observable in Raman scattering experiments")
    
    if activity['ir_active']:
        print("  ✓ Observable in infrared spectroscopy")
    
    # Show detailed rules if available
    if activity['selection_rules']:
        print("\nDetailed Symmetry Rules:")
        for j, rule in enumerate(activity['selection_rules']):
            if rule.get('notes'):
                for note in rule['notes']:
                    print(f"  • {note}")

## Visualization of Results

Let's create visualizations to better understand the symmetry analysis results:

In [ ]:
# Create comprehensive visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Energy level diagram
energies = results['unique_energies']
degeneracies = results['degeneracies']
irreps = results['irrep_decomposition']
activities = results['optical_activity']

# Plot energy levels with color coding for optical activity
for i, (energy, degen, irrep, activity) in enumerate(zip(energies, degeneracies, irreps, activities)):
    if activity['electric_dipole_allowed']:
        color = 'red'
        label = 'Bright (Optically Active)' if i == 0 else ''
    else:
        color = 'blue'
        label = 'Dark (Optically Forbidden)' if i == 0 else ''
    
    # Draw energy level
    ax1.hlines(energy, 0, 1, colors=color, linewidth=4, label=label)
    
    # Add labels
    ax1.text(1.1, energy, f'{irrep} (deg={degen})', 
             verticalalignment='center', fontsize=10)

ax1.set_xlim(-0.1, 2.5)
ax1.set_ylabel('Energy (eV)')
ax1.set_title('Exciton Energy Levels and Symmetries')
ax1.legend()
ax1.set_xticks([])
ax1.grid(True, alpha=0.3)

# 2. Optical activity summary
activity_counts = {'Optical': 0, 'Raman': 0, 'IR': 0, 'Forbidden': 0}

for activity in activities:
    if activity['electric_dipole_allowed']:
        activity_counts['Optical'] += 1
    if activity['raman_active']:
        activity_counts['Raman'] += 1
    if activity['ir_active']:
        activity_counts['IR'] += 1
    if not any([activity['electric_dipole_allowed'], 
                activity['raman_active'], 
                activity['ir_active']]):
        activity_counts['Forbidden'] += 1

activities_list = list(activity_counts.keys())
counts = list(activity_counts.values())
colors = ['red', 'green', 'orange', 'gray']

bars = ax2.bar(activities_list, counts, color=colors, alpha=0.7)
ax2.set_ylabel('Number of States')
ax2.set_title('Optical Activity Summary')
ax2.set_ylim(0, max(counts) + 1)
ax2.grid(True, alpha=0.3)

# Add value labels on bars
for bar, count in zip(bars, counts):
    if count > 0:
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                str(count), ha='center', va='bottom')

# 3. Degeneracy distribution
unique_degens, degen_counts = np.unique(degeneracies, return_counts=True)
ax3.bar(unique_degens, degen_counts, alpha=0.7, color='purple')
ax3.set_xlabel('Degeneracy')
ax3.set_ylabel('Number of Levels')
ax3.set_title('Degeneracy Distribution')
ax3.grid(True, alpha=0.3)

# 4. Energy distribution histogram
all_energies = results['exciton_energies']
ax4.hist(all_energies, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
ax4.axvline(np.mean(all_energies), color='red', linestyle='--', 
           label=f'Mean: {np.mean(all_energies):.3f} eV')
ax4.set_xlabel('Energy (eV)')
ax4.set_ylabel('Number of States')
ax4.set_title('Exciton Energy Distribution')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Save Analysis Results

The `save_analysis_results()` method saves the complete analysis to a text file for future reference:

In [ ]:
# Save the analysis results
output_filename = f'exciton_symmetry_Q{iQ}_analysis.txt'

print(f"Saving analysis results to: {output_filename}")
egt.save_analysis_results(results, output_filename)

print("\nResults saved successfully!")
print(f"\nFile contains:")
print("- Point group identification")
print("- Little group operations")
print("- Symmetry classes")
print("- Energy levels with irreducible representations")
print("- Optical selection rules")

# Display a preview of the saved file
if os.path.exists(output_filename):
    print(f"\nPreview of {output_filename}:")
    print("-" * 50)
    with open(output_filename, 'r') as f:
        lines = f.readlines()[:20]  # Show first 20 lines
        for line in lines:
            print(line.rstrip())
    print("...")
    print(f"(showing first 20 lines of {len(open(output_filename).readlines())} total lines)")